In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda:0')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [3]:
path = 'C:/Users/ideal/Downloads/jupyter/user_data/'

In [4]:
train_png = sorted(glob(path + 'train/*.png'))
test_png = sorted(glob(path + 'test/*.png'))

In [5]:
len(train_png), len(test_png)

(858, 215)

In [6]:
train_y = pd.read_csv(path +"train.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [7]:
label_unique

{'1': 0,
 '10-1': 1,
 '10-2': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [8]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
#     img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [9]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████████████████████████████████████████████████████████████████████████| 215/215 [00:00<00:00, 1080.38it/s]


In [10]:
np.save(path + 'train_imgs_384', np.array(train_imgs))
np.save(path + 'test_imgs_384', np.array(test_imgs))

In [11]:
train_imgs = np.load(path + 'train_imgs_384.npy')
test_imgs = np.load(path + 'test_imgs_384.npy')

In [12]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

train_meanR = np.mean([m[0] for m in meanRGB])/255
train_meanG = np.mean([m[1] for m in meanRGB])/255
train_meanB = np.mean([m[2] for m in meanRGB])/255

train_stdR = np.mean([s[0] for s in stdRGB])/255
train_stdG = np.mean([s[1] for s in stdRGB])/255
train_stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",train_meanR, train_meanG, train_meanB)
print("train 표준편차",train_stdR, train_stdG, train_stdB)

train 평균 0.587861452947292 0.5398018372012267 0.4853426659853918
train 표준편차 0.15058758283288234 0.15921522386293296 0.17031454681984776


In [13]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

test_meanR = np.mean([m[0] for m in meanRGB])/255
test_meanG = np.mean([m[1] for m in meanRGB])/255
test_meanB = np.mean([m[2] for m in meanRGB])/255

test_stdR = np.mean([s[0] for s in stdRGB])/255
test_stdG = np.mean([s[1] for s in stdRGB])/255
test_stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",test_meanR, test_meanG, test_meanB)
print("test 표준편차",test_stdR, test_stdG, test_stdB)

test 평균 0.5915704246815005 0.5468021681783177 0.49356994941872095
test 표준편차 0.15494109227168867 0.1642936360901455 0.1756015391157054


In [14]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [train_meanR, train_meanG, train_meanB],
                                     std = [train_stdR, train_stdG, train_stdB]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [test_meanR, test_meanG, test_meanB],
                                     std = [test_stdR, test_stdG, test_stdB])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('resnet34', pretrained=True, num_classes=11)
        if self.mode == 'test':
          self.model = timm.create_model('resnet34', pretrained=True, num_classes=11)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [15]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [16]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle = True)
batch_size = 8
epochs = 100
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-3)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet34-43635321.pth" to C:\Users\ideal/.cache\torch\hub\checkpoints\resnet34-43635321.pth


-----------------SAVE:1 epoch----------------
epoch : 1/100    time : 13s/1286s
TRAIN    loss : 1.62549    f1 : 0.46980
Val    loss : 0.83261    f1 : 0.73444
-----------------SAVE:2 epoch----------------
epoch : 2/100    time : 12s/1155s
TRAIN    loss : 0.74099    f1 : 0.74982
Val    loss : 0.52565    f1 : 0.82016
-----------------SAVE:3 epoch----------------
epoch : 3/100    time : 12s/1148s
TRAIN    loss : 0.44355    f1 : 0.85748
Val    loss : 0.37140    f1 : 0.87893
-----------------SAVE:4 epoch----------------
epoch : 4/100    time : 12s/1129s
TRAIN    loss : 0.37303    f1 : 0.88726
Val    loss : 0.36460    f1 : 0.89435
-----------------SAVE:5 epoch----------------
epoch : 5/100    time : 12s/1132s
TRAIN    loss : 0.32303    f1 : 0.89617
Val    loss : 0.24003    f1 : 0.95311
epoch : 6/100    time : 12s/1100s
TRAIN    loss : 0.27515    f1 : 0.91505
Val    loss : 0.22981    f1 : 0.90527
epoch : 7/100    time : 12s/1106s
TRAIN    loss : 0.20308    f1 : 0.94206
Val    loss : 0.20915   

epoch : 27/100    time : 12s/861s
TRAIN    loss : 0.02435    f1 : 0.99576
Val    loss : 0.19566    f1 : 0.93515
epoch : 28/100    time : 12s/855s
TRAIN    loss : 0.12828    f1 : 0.96459
Val    loss : 0.09855    f1 : 0.97065
epoch : 29/100    time : 12s/846s
TRAIN    loss : 0.06798    f1 : 0.98267
Val    loss : 0.31467    f1 : 0.89667
epoch : 30/100    time : 12s/823s
TRAIN    loss : 0.07544    f1 : 0.98132
Val    loss : 0.10229    f1 : 0.96336
epoch : 31/100    time : 13s/868s
TRAIN    loss : 0.09633    f1 : 0.97394
Val    loss : 0.15915    f1 : 0.92255
epoch : 32/100    time : 15s/1038s
TRAIN    loss : 0.12344    f1 : 0.96652
Val    loss : 0.17586    f1 : 0.94030
epoch : 33/100    time : 16s/1048s
TRAIN    loss : 0.07641    f1 : 0.97872
Val    loss : 0.14567    f1 : 0.96569
epoch : 34/100    time : 15s/986s
TRAIN    loss : 0.06917    f1 : 0.97956
Val    loss : 0.41461    f1 : 0.90472
epoch : 35/100    time : 15s/1002s
TRAIN    loss : 0.08374    f1 : 0.98146
Val    loss : 0.15504    f1

-----------------SAVE:24 epoch----------------
epoch : 24/100    time : 16s/1184s
TRAIN    loss : 0.01862    f1 : 0.99713
Val    loss : 0.07336    f1 : 0.98328
epoch : 25/100    time : 15s/1126s
TRAIN    loss : 0.06202    f1 : 0.98288
Val    loss : 0.30920    f1 : 0.91785
epoch : 26/100    time : 16s/1200s
TRAIN    loss : 0.07327    f1 : 0.98291
Val    loss : 0.11544    f1 : 0.97794
-----------------SAVE:27 epoch----------------
epoch : 27/100    time : 15s/1088s
TRAIN    loss : 0.04200    f1 : 0.98969
Val    loss : 0.13004    f1 : 0.98344
epoch : 28/100    time : 15s/1092s
TRAIN    loss : 0.06239    f1 : 0.97981
Val    loss : 0.18661    f1 : 0.94154
epoch : 29/100    time : 15s/1078s
TRAIN    loss : 0.10459    f1 : 0.97007
Val    loss : 0.18848    f1 : 0.93516
epoch : 30/100    time : 15s/1039s
TRAIN    loss : 0.08948    f1 : 0.97497
Val    loss : 0.15702    f1 : 0.96096
epoch : 31/100    time : 15s/1039s
TRAIN    loss : 0.06209    f1 : 0.98416
Val    loss : 0.22395    f1 : 0.94267
ep

epoch : 45/100    time : 12s/657s
TRAIN    loss : 0.02126    f1 : 0.99411
Val    loss : 0.15831    f1 : 0.96520
epoch : 46/100    time : 12s/633s
TRAIN    loss : 0.04367    f1 : 0.98946
Val    loss : 0.21052    f1 : 0.94577
epoch : 47/100    time : 12s/630s
TRAIN    loss : 0.03934    f1 : 0.99144
Val    loss : 0.10371    f1 : 0.96380
epoch : 48/100    time : 12s/620s
TRAIN    loss : 0.04756    f1 : 0.97996
Val    loss : 0.15375    f1 : 0.95150
epoch : 49/100    time : 12s/601s
TRAIN    loss : 0.07802    f1 : 0.97729
Val    loss : 0.20174    f1 : 0.96050
epoch : 50/100    time : 12s/589s
TRAIN    loss : 0.03909    f1 : 0.99007
Val    loss : 0.20609    f1 : 0.95262
epoch : 51/100    time : 12s/585s
TRAIN    loss : 0.03822    f1 : 0.99301
Val    loss : 0.22475    f1 : 0.95444
epoch : 52/100    time : 12s/566s
TRAIN    loss : 0.02858    f1 : 0.99156
Val    loss : 0.12776    f1 : 0.95772
epoch : 53/100    time : 12s/562s
TRAIN    loss : 0.03951    f1 : 0.99154
Val    loss : 0.27619    f1 : 

In [17]:
pred_ensemble = []
batch_size = 8
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [18]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [19]:
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

In [20]:
submission = pd.read_csv(path + "sample_submission.csv")
submission["label"] = f_result
submission

,file_name,label
0,001.png,1
1,002.png,2
2,003.png,1
3,004.png,6
4,005.png,8
...,...,...
210,211.png,5
211,212.png,8
212,213.png,3
213,214.png,6


In [21]:
submission.to_csv(path + "submit.csv", index = False)

In [22]:
# d9249@kyonggi.ac.kr

from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'C:/Users/ideal/Downloads/jupyter/user_data/submit.csv', 
'02438df9bd0f6300dae6ddea845e7e01d2cb1881849c166bfce504164e1507d5', 
'235896', 
'iDeal', 
'test' )

{'isSubmitted': False, 'detail': 'Over max submission count of Daily. 일일 제출 가능한 최대 횟수가 초과 되었습니다.'}
